<a href="https://colab.research.google.com/github/sayakpaul/EvoNorms-in-TensorFlow-2/blob/master/Mini_Inception_BN_ReLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Which GPU? 
!nvidia-smi

Sat Apr 18 07:34:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# TensorFlow imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [0]:
# Other imports
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np

In [0]:
# Set the random seeds
tf.random.set_seed(666)
np.random.seed(666)

In [0]:
# Set up wandb for easy experiment tracking
!pip install wandb -q
import wandb
wandb.login()

In [6]:
# Load and preprocess CIFAR10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
print(X_train.shape, X_test.shape)

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [0]:
# Implementation comes from http://pyimg.co/mac01
def minigooglenet_functional(width, height, depth, classes):
	def conv_module(x, K, kX, kY, stride, chanDim, padding="same"):
		# define a CONV => BN => RELU pattern
		x = Conv2D(K, (kX, kY), strides=stride, padding=padding)(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation("relu")(x)

		# return the block
		return x

	def inception_module(x, numK1x1, numK3x3, chanDim):
		# define two CONV modules, then concatenate across the
		# channel dimension
		conv_1x1 = conv_module(x, numK1x1, 1, 1, (1, 1), chanDim)
		conv_3x3 = conv_module(x, numK3x3, 3, 3, (1, 1), chanDim)
		x = concatenate([conv_1x1, conv_3x3], axis=chanDim)

		# return the block
		return x

	def downsample_module(x, K, chanDim):
		# define the CONV module and POOL, then concatenate
		# across the channel dimensions
		conv_3x3 = conv_module(x, K, 3, 3, (2, 2), chanDim,
			padding="valid")
		pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
		x = concatenate([conv_3x3, pool], axis=chanDim)

		# return the block
		return x

	# initialize the input shape to be "channels last" and the
	# channels dimension itself
	inputShape = (height, width, depth)
	chanDim = -1

	# define the model input and first CONV module
	inputs = Input(shape=inputShape)
	x = conv_module(inputs, 96, 3, 3, (1, 1), chanDim)

	# two Inception modules followed by a downsample module
	x = inception_module(x, 32, 32, chanDim)
	x = inception_module(x, 32, 48, chanDim)
	x = downsample_module(x, 80, chanDim)

	# four Inception modules followed by a downsample module
	x = inception_module(x, 112, 48, chanDim)
	x = inception_module(x, 96, 64, chanDim)
	x = inception_module(x, 80, 80, chanDim)
	x = inception_module(x, 48, 96, chanDim)
	x = downsample_module(x, 96, chanDim)

	# two Inception modules followed by global POOL and dropout
	x = inception_module(x, 176, 160, chanDim)
	x = inception_module(x, 176, 160, chanDim)
	x = AveragePooling2D((7, 7))(x)
	x = Dropout(0.5)(x)

	# softmax classifier
	x = Flatten()(x)
	x = Dense(classes)(x)
	x = Activation("softmax")(x)

	# create the model
	model = Model(inputs, x, name="minigooglenet")

	# return the constructed network architecture
	return model

In [0]:
# Hyperparams
BATCH_SIZE = 128
EPOCHS = 60

In [0]:
# Import wandb's Keras callback
from wandb.keras import WandbCallback

In [11]:
# Initialize wandb
wandb.init(project="evonorm-tf2", id="bn-relu-adam-no-aug")

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"])
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 20s 50ms/step - loss: 1.2624 - accuracy: 0.5445 - val_loss: 2.0476 - val_accuracy: 0.3174
Epoch 2/60
391/391 [==============================] - 19s 47ms/step - loss: 0.7987 - accuracy: 0.7180 - val_loss: 1.2319 - val_accuracy: 0.6011
Epoch 3/60
391/391 [==============================] - 19s 47ms/step - loss: 0.6086 - accuracy: 0.7892 - val_loss: 0.7520 - val_accuracy: 0.7443
Epoch 4/60
391/391 [==============================] - 18s 47ms/step - loss: 0.4908 - accuracy: 0.8306 - val_loss: 0.7714 - val_accuracy: 0.7509
Epoch 5/60
391/391 [==============================] - 18s 47ms/step - loss: 0.4092 - accuracy: 0.8596 - val_loss: 0.9075 - val_accuracy: 0.6802
Epoch 6/60
391/391 [==============================] - 19s 47ms/step - loss: 0.3421 - accuracy: 0.8809 - val_loss: 0.6310 - val_accuracy: 0.7970
Epoch 7/60
391/391 [==============================] - 18s 47ms/step - loss: 0.2802 - accuracy: 0.9023 - val_loss: 0.7355 - val_accuracy:

In [0]:
# Let's try one-hot encoding the labels and then see
y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_test_ohe = tf.keras.utils.to_categorical(y_test)

In [13]:
# Initialize wandb
wandb.init(project="evonorm-tf2", id="bn-relu-adam-no-aug-ohe")

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
	metrics=["accuracy"])
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 20s 50ms/step - loss: 1.2584 - accuracy: 0.5458 - val_loss: 1.9984 - val_accuracy: 0.3539
Epoch 2/60
391/391 [==============================] - 19s 49ms/step - loss: 0.7739 - accuracy: 0.7303 - val_loss: 1.2209 - val_accuracy: 0.6240
Epoch 3/60
391/391 [==============================] - 19s 49ms/step - loss: 0.5982 - accuracy: 0.7944 - val_loss: 0.8917 - val_accuracy: 0.6979
Epoch 4/60
391/391 [==============================] - 19s 49ms/step - loss: 0.4890 - accuracy: 0.8312 - val_loss: 0.7485 - val_accuracy: 0.7473
Epoch 5/60
391/391 [==============================] - 19s 48ms/step - loss: 0.4051 - accuracy: 0.8615 - val_loss: 0.9135 - val_accuracy: 0.7051
Epoch 6/60
391/391 [==============================] - 19s 48ms/step - loss: 0.3343 - accuracy: 0.8843 - val_loss: 0.8170 - val_accuracy: 0.7421
Epoch 7/60
391/391 [==============================] - 19s 48ms/step - loss: 0.2810 - accuracy: 0.9036 - val_loss: 1.2142 - val_accuracy:

In [15]:
# Initialize wandb
wandb.init(project="evonorm-tf2", id="bn-relu-sgd-no-aug-one-hot")

# Let's try changing the optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(X_train, y_train_ohe,
                    validation_data=(X_test, y_test_ohe),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=[WandbCallback()])

Epoch 1/60
391/391 [==============================] - 19s 50ms/step - loss: 1.4082 - accuracy: 0.4863 - val_loss: 2.1256 - val_accuracy: 0.2715
Epoch 2/60
391/391 [==============================] - 19s 48ms/step - loss: 0.9548 - accuracy: 0.6590 - val_loss: 1.4909 - val_accuracy: 0.5194
Epoch 3/60
391/391 [==============================] - 19s 48ms/step - loss: 0.7681 - accuracy: 0.7282 - val_loss: 0.9395 - val_accuracy: 0.6679
Epoch 4/60
391/391 [==============================] - 19s 48ms/step - loss: 0.6520 - accuracy: 0.7734 - val_loss: 1.1169 - val_accuracy: 0.6309
Epoch 5/60
391/391 [==============================] - 19s 48ms/step - loss: 0.5607 - accuracy: 0.8044 - val_loss: 1.0874 - val_accuracy: 0.6387
Epoch 6/60
391/391 [==============================] - 19s 48ms/step - loss: 0.4810 - accuracy: 0.8316 - val_loss: 0.9093 - val_accuracy: 0.7136
Epoch 7/60
391/391 [==============================] - 19s 48ms/step - loss: 0.4131 - accuracy: 0.8573 - val_loss: 0.8429 - val_accuracy:

In [0]:
# Let's try with data augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=18, 
    zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, 
    shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [17]:
# Initialize wandb
wandb.init(project="evonorm-tf2", id="bn-relu-sgd-aug-ohe")

# Let's try changing the optimizer
opt = tf.keras.optimizers.SGD(lr=1e-2, momentum=0.9, decay=1e-2 / EPOCHS)

# Compile and train this model
model = minigooglenet_functional(32, 32, 3, 10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
history = model.fit(aug.flow(X_train, y_train_ohe),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
	validation_data=(X_test, y_test_ohe),
	validation_steps=X_test.shape[0] // BATCH_SIZE,
	epochs=EPOCHS,
	callbacks=[WandbCallback()])

Epoch 1/60
390/390 [==============================] - 14s 35ms/step - loss: 1.9018 - accuracy: 0.3034 - val_loss: 1.6943 - val_accuracy: 0.3652
Epoch 2/60
390/390 [==============================] - 13s 32ms/step - loss: 1.5763 - accuracy: 0.4292 - val_loss: 2.2856 - val_accuracy: 0.2852
Epoch 3/60
390/390 [==============================] - 12s 32ms/step - loss: 1.4258 - accuracy: 0.4808 - val_loss: 1.8869 - val_accuracy: 0.4210
Epoch 4/60
390/390 [==============================] - 13s 33ms/step - loss: 1.3485 - accuracy: 0.5188 - val_loss: 1.1502 - val_accuracy: 0.5886
Epoch 5/60
390/390 [==============================] - 13s 32ms/step - loss: 1.2429 - accuracy: 0.5530 - val_loss: 1.2371 - val_accuracy: 0.5524
Epoch 6/60
390/390 [==============================] - 12s 32ms/step - loss: 1.1927 - accuracy: 0.5780 - val_loss: 1.1611 - val_accuracy: 0.5992
Epoch 7/60
390/390 [==============================] - 13s 32ms/step - loss: 1.1465 - accuracy: 0.5930 - val_loss: 1.1244 - val_accuracy: